In [8]:
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import os
from dotenv import load_dotenv
import time

# FMP News API Code

In [9]:
load_dotenv(r"api.env")

fmp_api_key = os.getenv("fmp_api_key")
if not fmp_api_key:
    raise ValueError("No API key set for fmp_api_key in .env file")

print("API key loaded successfully from .env file.")

API key loaded successfully from .env file.


In [10]:
stock_news = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

press_releases = [
    {"company": "AMZN", "ipo_date": "1997-05-15"},
    {"company": "AAPL", "ipo_date": "1980-12-12"},
    {"company": "GOOG", "ipo_date": "2004-08-19"},
    {"company": "MSFT", "ipo_date": "1986-03-13"},
    {"company": "META", "ipo_date": "2012-05-18"},
    {"company": "NVDA", "ipo_date": "1999-01-22"}
]

historical_social_sentiment = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

In [11]:
current_date =date.today()

for news in stock_news:
    url = f"https://financialmodelingprep.com/api/v3/stock_news?"

    page = 0
    limit = 1000
    all_news = []

    while True:
        params = {
        "apikey" : fmp_api_key,
        "tickers" : news['symbol'],
        "page": page,
        'from': news['ipo_date'],
        'to': current_date,
        'limit': limit
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {news['symbol']} on page {page}: {e}")
            break

        data = response.json()

        if not data:
            break


        all_news.extend(data)

        # if len(data) < limit:
        #     break

        page += 1

    csv_file = f"{news['symbol']}_news_data.csv"
    headers = ["published_date", "Headline", "Brief", "URL"]
    formatted_data = []

    if all_news:
        for record in all_news:

            formats = ['%Y-%m-%d %I:%M:%S %p', '%Y-%m-%d %H:%M:%S']

            if 'publishedDate' in record:
                for fmt in formats:
                    try:
                        record['publishedDate'] = datetime.strptime(record['publishedDate'], fmt).strftime('%d-%m-%Y')
                        break
                    except ValueError as e:
                        continue
                        # print(f"Error processing date {record['date']}: {e}")

            formatted_data.append({
            "published_date": record.get("publishedDate", ""),
            "Headline": record.get("title", ""),
            "Brief": record.get("text", ""),
            "URL": record.get("url", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {news['symbol']} written to {csv_file} with {len(formatted_data)} articles!")
    else:
        print(f"No data found for {news['symbol']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

Data for AMZN written to AMZN_news_data.csv with 29074 articles!
Data for AAPL written to AAPL_news_data.csv with 26284 articles!
Data for GOOG written to GOOG_news_data.csv with 17631 articles!
Data for MSFT written to MSFT_news_data.csv with 16622 articles!
Data for META written to META_news_data.csv with 18255 articles!
Data for NVDA written to NVDA_news_data.csv with 18739 articles!
All stock data successfully written to their respective CSV files.


In [12]:
for press_release in press_releases:
    url = f"https://financialmodelingprep.com/api/v3/press-releases/{press_release['company']}?"

    page = 0
    all_press_releases = []

    max_pages = 200

    while page < max_pages:
        params = {
        "apikey" : fmp_api_key,
        "page": page
        }

        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {press_release['company']} on page {page}: {e}")
            break

        time.sleep(1)

        data = response.json()
        print(f"Fetching data for {press_release['company']} - Page {page}...")

        if not data:
            break


        all_press_releases.extend(data)

        page += 1

    csv_file = f"{press_release['company']}_press_release_data.csv"
    headers = ["release_date", "Headline", "Brief"]
    formatted_data = []

    if all_press_releases:
        for record in all_press_releases:

            formats = ['%Y-%m-%d %I:%M:%S %p', '%Y-%m-%d %H:%M:%S']

            if 'date' in record:
                for fmt in formats:
                    try:
                        record['date'] = datetime.strptime(record['date'], fmt).strftime('%d-%m-%Y')
                        break
                    except ValueError as e:
                        continue

            formatted_data.append({
            "release_date": record.get("date", ""),
            "Headline": record.get("title", ""),
            "Brief": record.get("text", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {press_release['company']} written to {csv_file} with {len(formatted_data)} press releases!")
    else:
        print(f"No data found for {press_release['company']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

Fetching data for AMZN - Page 0...
Fetching data for AMZN - Page 1...
Fetching data for AMZN - Page 2...
Fetching data for AMZN - Page 3...
Fetching data for AMZN - Page 4...
Fetching data for AMZN - Page 5...
Fetching data for AMZN - Page 6...
Fetching data for AMZN - Page 7...
Fetching data for AMZN - Page 8...
Fetching data for AMZN - Page 9...
Fetching data for AMZN - Page 10...
Fetching data for AMZN - Page 11...
Fetching data for AMZN - Page 12...
Fetching data for AMZN - Page 13...
Fetching data for AMZN - Page 14...
Fetching data for AMZN - Page 15...
Fetching data for AMZN - Page 16...
Fetching data for AMZN - Page 17...
Fetching data for AMZN - Page 18...
Fetching data for AMZN - Page 19...
Fetching data for AMZN - Page 20...
Fetching data for AMZN - Page 21...
Fetching data for AMZN - Page 22...
Fetching data for AMZN - Page 23...
Fetching data for AMZN - Page 24...
Fetching data for AMZN - Page 25...
Fetching data for AMZN - Page 26...
Fetching data for AMZN - Page 27...
Fe

In [ ]:
# Load the existing CSV file
csv_filename = "news_data.csv"
df = pd.read_csv(csv_filename)

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score > 0.05:
        return "Positive"
    elif score < -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to each row
df["Sentiment"] = df.apply(lambda row: get_sentiment(f"{row['Headline']} {row['Brief']}"), axis=1)

# Save back to the same CSV file (overwrite with sentiment added)
df.to_csv(csv_filename, index=False)
print(f"Sentiment appended to {csv_filename} successfully!")